## 建立資料集
建立一個很簡單乘法器作為我們的迴歸範例。一個輸入對應一個輸出，其輸出為輸入的倍數。舉例來說，輸入 4 其輸出為 8。

In [85]:
import numpy as np

X = np.arange(1, 500, 1)
y = np.arange(2, 1000, 2)

## 搭建神經網路模型

In [86]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


In [87]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model = Sequential()
model.add(Dense(1, input_shape=[1]))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=2), loss=keras.losses.mean_squared_error)

In [88]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.fit(X, y, epochs=100)

Epoch 1/100
16/16 [==============================] - 0s 3ms/step - loss: 42916.7969
Epoch 2/100
16/16 [==============================] - 0s 2ms/step - loss: 9716.6758
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 2083.0762
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 472.8735
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 94.4096
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 17.7075
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 5.1454
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 1.0494
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.3086
Epoch 10/100
16/16 [==============================] - 0s 1ms/step - loss: 0.1093
Epoch 11/100
16/16 [==============================] - 0s 1ms/step - loss: 0.0546
Epoch 12/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0371
Epoch 13/100
16/16 [===

## 測試模型

In [90]:
model.predict([10])

array([[20.]], dtype=float32)

## 儲存模型
打包成 `.h5` 可以將模型儲存起來下次，Python 要執行讀入模型即可立即預測。另外透過 TFLiteConverter 可以將 Python 訓練好的模型打包成 `.tflite` 格式，並提供手機設備進行預測。

In [91]:
model.save('regression.h5')

In [92]:
import tensorflow as tf
model = tf.keras.models.load_model('regression.h5')
converter =tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("regression.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_yifk_gg/assets


832

## TFLite Interpreter 測試模型

In [19]:
import tensorflow as tf
import numpy as np

interpreter = tf.lite.Interpreter(model_path='regression.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

input_data = np.array([[100]], dtype=np.float32)
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()

print(interpreter.get_tensor(output_index))

[[200.]]
